In [11]:
# ✅ [0단계] 데이터 준비: 로컬 zip 파일 압축 해제
import os
import zipfile
import shutil

# 🚨🚨🚨 경로 수정 🚨🚨🚨
# 현재 노트북 파일(CUDA.ipynb)이 있는 'model_test' 폴더를 기준으로 상대 경로를 지정합니다.
zip_files_dir = "image_zip"
unzip_target_dir = "dataset"

# 만약 타겟 폴더가 이미 있다면 삭제 (이전 실행 결과 초기화)
if os.path.exists(unzip_target_dir):
    shutil.rmtree(unzip_target_dir)
os.makedirs(unzip_target_dir, exist_ok=True)

print(f"압축 해제 시작... (대상 폴더: '{unzip_target_dir}')")

# zip 폴더 내의 모든 zip 파일에 대해 반복
for file_name in os.listdir(zip_files_dir):
    if file_name.endswith('.zip'):
        zip_file_path = os.path.join(zip_files_dir, file_name)

        # 클래스 이름은 zip 파일명에서 확장자를 제거하여 사용 (예: 감바2.zip -> 감바2)
        class_name = os.path.splitext(file_name)[0]
        class_dir = os.path.join(unzip_target_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # zip 파일 압축 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(class_dir)
        print(f"  ✅ '{file_name}' -> '{class_dir}' 폴더에 압축 해제 완료")

print("모든 압축 파일 해제 완료!")

압축 해제 시작... (대상 폴더: 'dataset')
  ✅ '감바2.zip' -> 'dataset\감바2' 폴더에 압축 해제 완료
  ✅ '숯불2.zip' -> 'dataset\숯불2' 폴더에 압축 해제 완료
  ✅ '양념2.zip' -> 'dataset\양념2' 폴더에 압축 해제 완료
  ✅ '파스타2.zip' -> 'dataset\파스타2' 폴더에 압축 해제 완료
  ✅ '후라이드치킨2.zip' -> 'dataset\후라이드치킨2' 폴더에 압축 해제 완료
모든 압축 파일 해제 완료!


In [12]:
# ✅ [데이터 클리닝 Step] macOS 메타데이터 파일 삭제
# ==============================================================================
import os
import shutil

# 데이터셋의 루트 폴더 경로 (압축 해제된 폴더)
clean_target_dir = "dataset"

print(f"데이터 클리닝 시작... (대상 폴더: '{clean_target_dir}')")

# 폴더 내의 모든 하위 폴더와 파일을 순회
for root, dirs, files in os.walk(clean_target_dir):
    # 1. '__MACOSX' 폴더 삭제
    if '__MACOSX' in dirs:
        macosx_path = os.path.join(root, '__MACOSX')
        print(f"  - [폴더 삭제] '{macosx_path}'")
        shutil.rmtree(macosx_path)

    # 2. '._'로 시작하는 파일 삭제
    for file in files:
        if file.startswith('._'):
            file_path = os.path.join(root, file)
            try:
                print(f"  - [파일 삭제] '{file_path}'")
                os.remove(file_path)
            except OSError as e:
                print(f"  - [삭제 실패] '{file_path}': {e}")

print("✅ 데이터 클리닝 완료!")

데이터 클리닝 시작... (대상 폴더: 'dataset')
  - [폴더 삭제] 'dataset\감바2\__MACOSX'
  - [폴더 삭제] 'dataset\숯불2\__MACOSX'
  - [폴더 삭제] 'dataset\양념2\__MACOSX'
  - [폴더 삭제] 'dataset\파스타2\__MACOSX'
  - [폴더 삭제] 'dataset\후라이드치킨2\__MACOSX'
✅ 데이터 클리닝 완료!


In [13]:
# ✅ [1차 학습 Step 1] 환경 설정 및 라이브러리 로드

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
import numpy as np
from tqdm.notebook import tqdm
import time
import glob
from collections import Counter
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from torch.utils.data.sampler import WeightedRandomSampler
import random
import copy
from PIL import Image  # Pillow 라이브러리는 예측 함수에서도 사용되므로 import

print("✅ 필수 라이브러리 로드 완료")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ 현재 사용 디바이스: {device}")


# =============================================

✅ 필수 라이브러리 로드 완료
⚙️ 현재 사용 디바이스: cuda


In [14]:
# ✅ [1차 학습 Step 2] 데이터 로딩 및 불균형 처리 (Class Weight & Oversampling)

# 🚨🚨🚨 데이터 폴더 경로를 위에서 압축 해제한 로컬 경로로 수정 🚨🚨🚨
data_dir = unzip_target_dir  # "dataset" 폴더를 사용

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"⚠️ 경로를 찾을 수 없습니다: {data_dir}. 맨 위 압축 해제 셀이 정상적으로 실행되었는지 확인해주세요.")

# EfficientNet B0를 위한 전처리 설정
input_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# 데이터 증강 Transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

print("데이터셋 로딩 중...")

# 전체 데이터셋 로딩
full_dataset = datasets.ImageFolder(data_dir)
class_names = full_dataset.classes
num_classes = len(class_names)

# 훈련:검증 데이터 분할 (80:20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"✅ 클래스 목록: {class_names}")
print(f"🔢 전체 클래스 수: {num_classes}")
print(f"🖼️ 전체 이미지 수: {len(full_dataset)}")
print(f"📏 훈련 데이터 크기: {len(train_data)}개, 검증 데이터 크기: {len(val_data)}개")


# Custom Dataset 클래스: 분할된 데이터에 전처리 적용
class CustomDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        original_index = self.subset.indices[index]
        x, y = self.subset.dataset[original_index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)


# 전처리 적용된 데이터셋
image_datasets = {
    'train': CustomDataset(train_data, transform=data_transforms['train']),
    'val': CustomDataset(val_data, transform=data_transforms['val'])
}

# ----------------------------------------------------
# 🚨 불균형 처리 1: 클래스별 가중치 계산 (Loss Weighting)
# ----------------------------------------------------
train_labels = [full_dataset.targets[i] for i in train_data.indices]
class_counts = Counter(train_labels)
print(f"🔎 훈련 데이터 클래스별 개수: {class_counts}")

# 각 클래스의 가중치 (총 샘플 수 / 클래스별 샘플 수)
num_samples = len(train_labels)
class_weights = [num_samples / class_counts[i] for i in range(num_classes)]
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"⚖️ 손실 함수에 적용될 클래스 가중치: {class_weights_tensor.cpu().numpy()}")

# ----------------------------------------------------
# 🚨 불균형 처리 2: 오버샘플링을 위한 WeightedRandomSampler 적용
# ----------------------------------------------------
sample_weights = [class_weights[label] for label in train_labels]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)
print("♻️ 훈련 데이터에 WeightedRandomSampler (오버샘플링) 적용 완료.")


batch_size = 32

# dataloaders_dict = {
#     'train': DataLoader(image_datasets['train'], batch_size=batch_size, sampler=sampler, num_workers=2),
#     'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=2)
# }

dataloaders_dict = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, sampler=sampler, num_workers=0),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=0)
}

데이터셋 로딩 중...
✅ 클래스 목록: ['감바2', '숯불2', '양념2', '파스타2', '후라이드치킨2']
🔢 전체 클래스 수: 5
🖼️ 전체 이미지 수: 2555
📏 훈련 데이터 크기: 2044개, 검증 데이터 크기: 511개
🔎 훈련 데이터 클래스별 개수: Counter({4: 567, 3: 550, 1: 388, 2: 352, 0: 187})
⚖️ 손실 함수에 적용될 클래스 가중치: [10.930481   5.268041   5.806818   3.7163637  3.6049383]
♻️ 훈련 데이터에 WeightedRandomSampler (오버샘플링) 적용 완료.


In [15]:
# ✅ [1차 학습 Step 3] EfficientNet-B0 모델 로드 및 전이 학습 설정

print("\n EfficientNet-B0 모델 로드 중...")
model_ft = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# 특징 추출 레이어 동결 (Freeze)
for param in model_ft.parameters():
    param.requires_grad = False

# 최종 분류 레이어 (Classifier) 변경
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

print("✅ 모델 설정 완료")
print(f"최종 분류 클래스 수: {num_classes}")


 EfficientNet-B0 모델 로드 중...
✅ 모델 설정 완료
최종 분류 클래스 수: 5


In [16]:
# ✅ [1차 학습 Step 4] 모델 훈련 함수 및 실행 (고급 전략 적용)

# 🚨 손실 함수 설정 시 class_weights 적용
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
# 1차 학습은 마지막 분류 레이어만 학습 (Freeze 해제된 파라미터만)
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001)

# ----------------------------------------------------
# 🚨 고급 데이터 증강: MixUp 구현 함수
# ----------------------------------------------------
MIXUP_ALPHA = 0.4


def mixup_data(x, y, alpha=1.0, device='cuda'):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def train_model(model, dataloaders, criterion, optimizer, num_epochs=25,
                checkpoint_path='efficientnet_best_balanced.pth'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_f1 = -1.0  # 🚨 최적 모델 기준을 Macro F1 Score로 설정

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            all_labels = []
            all_preds = []

            for inputs, labels in tqdm(dataloaders[phase], desc=f'{phase} Phase'):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        # 🚨 50% 확률로 MixUp 적용
                        if random.random() < 0.5:
                            mixed_inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, MIXUP_ALPHA, device)
                            outputs = model(mixed_inputs)
                            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                    else:  # Validation Phase
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # 통계 계산
                    if phase == 'val':
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f}', end=' ')

            if phase == 'val':
                # 🚨 성능 모니터링: Macro F1 Score 및 Accuracy 계산
                current_acc = accuracy_score(all_labels, all_preds)
                current_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

                # 클래스별 정확도 계산
                cm = confusion_matrix(all_labels, all_preds)
                with np.errstate(divide='ignore', invalid='ignore'):
                    class_accuracy = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

                print(f'Acc: {current_acc:.4f} Macro F1: {current_f1:.4f}')
                print('--- 클래스별 정확도 ---')
                for i, acc in enumerate(class_accuracy):
                    print(f'  {class_names[i]}: {acc:.4f}', end=' ')
                print('\n-----------------------')

                # 🚨 최적 모델 저장 기준: Macro F1 Score가 가장 높을 때 저장
                if current_f1 > best_val_f1:
                    best_val_f1 = current_f1
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(best_model_wts, checkpoint_path)
                    print(f"⭐ 최적 모델 저장됨: {checkpoint_path} (Macro F1: {best_val_f1:.4f})")

    time_elapsed = time.time() - since
    print(f'\n🌟 훈련 완료! 총 소요 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'🏆 최적 검증 Macro F1: {best_val_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model


# 훈련 실행
num_epochs = 50
model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs,
                       checkpoint_path='efficientnet_best_balanced.pth')


Epoch 1/50
----------


train Phase:   0%|          | 0/64 [00:00<?, ?it/s]

train Loss: 0.8963 

val Phase:   0%|          | 0/16 [00:00<?, ?it/s]

val Loss: 0.4235 Acc: 0.9276 Macro F1: 0.9194
--- 클래스별 정확도 ---
  감바2: 0.9322   숯불2: 0.8857   양념2: 1.0000   파스타2: 0.9481   후라이드치킨2: 0.9021 
-----------------------
⭐ 최적 모델 저장됨: efficientnet_best_balanced.pth (Macro F1: 0.9194)

Epoch 2/50
----------


...

val Loss: 0.1859 Acc: 0.9393 Macro F1: 0.9312
--- 클래스별 정확도 ---
  감바2: 0.9661   숯불2: 0.8952   양념2: 0.9855   파스타2: 0.9481   후라이드치킨2: 0.9301 
-----------------------

Epoch 50/50
----------


train Phase:   0%|          | 0/64 [00:00<?, ?it/s]

train Loss: 0.4643 

val Phase:   0%|          | 0/16 [00:00<?, ?it/s]

val Loss: 0.1606 Acc: 0.9628 Macro F1: 0.9572
--- 클래스별 정확도 ---
  감바2: 0.9661   숯불2: 0.9429   양념2: 0.9710   파스타2: 0.9778   후라이드치킨2: 0.9580 
-----------------------

🌟 훈련 완료! 총 소요 시간: 6분 59초
🏆 최적 검증 Macro F1: 0.9622


In [17]:
# ✅ [1차 학습 Step 5] 최종 모델 가중치 저장 (선택 사항)
final_model_path = 'efficientnet_final_balanced.pth'
torch.save(model_ft.state_dict(), final_model_path)
print(f"✅ 최종 모델 가중치 저장 완료: {final_model_path}")


# 🧪 모델 예측 함수 정의 (1차 학습 후 테스트 시 재사용)

def load_model_for_prediction(num_classes, model_path, device=device):
    model = models.efficientnet_b0(weights=None)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        print(f"✅ 모델 로드 완료: {model_path}")
    except Exception as e:
        print(f"❌ 가중치 파일을 로드할 수 없습니다. 경로 또는 파일명({model_path})을 확인하세요. 오류: {e}")
        return None
    return model


def preprocess_and_predict(image_path, model, transform, device):
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"⚠️  이미지 로드 중 오류 발생 ({image_path}): {e}")
        return -1, 0.0
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_batch)
        probabilities = torch.nn.functional.softmax(output, dim=1)[0]
        confidence, predicted_class = torch.max(probabilities, 0)
    return predicted_class.item(), confidence.item()


predict_transforms = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

✅ 최종 모델 가중치 저장 완료: efficientnet_final_balanced.pth


In [18]:
# 🧪 [1차 학습 후 테스트] - 1차 학습 최적 모델 테스트

# 🚨🚨🚨 예측할 단일 이미지 파일의 로컬 경로를 여기에 입력하세요! 🚨🚨🚨
# 예시: test_image_path_1st = "path/to/your/test/image.jpg"
test_image_path_1st = "테스트할_이미지_경로를_여기에_입력하세요.jpg"

print("\n" + "=" * 80)
print(f"🧪 1차 학습 최적 모델 테스트: {os.path.basename(test_image_path_1st)}")
print("=" * 80)

loaded_model_1st = load_model_for_prediction(num_classes, model_path='efficientnet_best_balanced.pth')

if loaded_model_1st and os.path.exists(test_image_path_1st):
    predicted_class, confidence = preprocess_and_predict(test_image_path_1st, loaded_model_1st, predict_transforms,
                                                         device)
    if predicted_class != -1:
        pred_class = class_names[predicted_class]
        conf_percent = confidence * 100
        print(f"✅ 예측 클래스: {pred_class}")
        print(f"✅ 신뢰도: {conf_percent:.2f}%")
    else:
        print("⚠️ 예측 함수 실행 중 오류가 발생했습니다.")
elif not os.path.exists(test_image_path_1st):
    print(f"⚠️ 테스트 이미지를 찾을 수 없습니다: {test_image_path_1st}")
print("=" * 80)


🧪 1차 학습 최적 모델 테스트: 테스트할_이미지_경로를_여기에_입력하세요.jpg
✅ 모델 로드 완료: efficientnet_best_balanced.pth
⚠️ 테스트 이미지를 찾을 수 없습니다: 테스트할_이미지_경로를_여기에_입력하세요.jpg


In [19]:
# 🧪 2차 학습: Fine-tuning 준비

print("\n✨ Fine-tuning 준비: 전체 모델 계층 동결 해제")

for param in model_ft.parameters():
    param.requires_grad = True

optimizer_ft_fine = optim.Adam(model_ft.parameters(), lr=0.00001)
criterion_fine = nn.CrossEntropyLoss(weight=class_weights_tensor)

print("최초 10개 계층의 require_grad 상태 확인:")
for name, param in list(model_ft.named_parameters())[:10]:
    print(f"{name}: requires_grad={param.requires_grad}")
print("✅ 2차 Fine-tuning 준비 완료. 모든 계층이 학습됩니다. (Class Weights 적용)")

print("\n" + "=" * 80)
print("🚀 2차 Fine-tuning (추가 학습) 시작")
print("=" * 80)
try:
    model_ft.load_state_dict(torch.load('efficientnet_best_balanced.pth', map_location=device))
    print("✅ 1차 학습에서 저장된 최적 가중치로 Fine-tuning 시작!")
except FileNotFoundError:
    print("⚠️ 1차 학습 파일(efficientnet_best_balanced.pth)을 찾을 수 없습니다. 현재 모델 상태로 Fine-tuning을 진행합니다.")

num_epochs_fine = 30
model_ft_fine_tuned = train_model(
    model_ft,
    dataloaders_dict,
    criterion_fine,
    optimizer_ft_fine,
    num_epochs=num_epochs_fine,
    checkpoint_path='efficientnet_fine_tuned_best_balanced.pth'
)
print("✅ 2차 Fine-tuning 완료")


✨ Fine-tuning 준비: 전체 모델 계층 동결 해제
최초 10개 계층의 require_grad 상태 확인:
features.0.0.weight: requires_grad=True
features.0.1.weight: requires_grad=True
features.0.1.bias: requires_grad=True
features.1.0.block.0.0.weight: requires_grad=True
features.1.0.block.0.1.weight: requires_grad=True
features.1.0.block.0.1.bias: requires_grad=True
features.1.0.block.1.fc1.weight: requires_grad=True
features.1.0.block.1.fc1.bias: requires_grad=True
features.1.0.block.1.fc2.weight: requires_grad=True
features.1.0.block.1.fc2.bias: requires_grad=True
✅ 2차 Fine-tuning 준비 완료. 모든 계층이 학습됩니다. (Class Weights 적용)

🚀 2차 Fine-tuning (추가 학습) 시작
✅ 1차 학습에서 저장된 최적 가중치로 Fine-tuning 시작!

Epoch 1/30
----------


train Phase:   0%|          | 0/64 [00:00<?, ?it/s]

train Loss: 0.4279 

val Phase:   0%|          | 0/16 [00:00<?, ?it/s]

val Loss: 0.1468 Acc: 0.9667 Macro F1: 0.9613
--- 클래스별 정확도 ---
  감바2: 0.9661   숯불2: 0.9333   양념2: 0.9855   파스타2: 0.9704   후라이드치킨2: 0.9790 
-----------------------
⭐ 최적 모델 저장됨: efficientnet_fine_tuned_best_balanced.pth (Macro F1: 0.9613)

Epoch 2/30
----------


...

val Loss: 0.1104 Acc: 0.9667 Macro F1: 0.9628
--- 클래스별 정확도 ---
  감바2: 0.9831   숯불2: 0.9238   양념2: 1.0000   파스타2: 0.9630   후라이드치킨2: 0.9790 
-----------------------

Epoch 30/30
----------


train Phase:   0%|          | 0/64 [00:00<?, ?it/s]

train Loss: 0.2853 

val Phase:   0%|          | 0/16 [00:00<?, ?it/s]

val Loss: 0.0915 Acc: 0.9746 Macro F1: 0.9706
--- 클래스별 정확도 ---
  감바2: 1.0000   숯불2: 0.9238   양념2: 1.0000   파스타2: 0.9778   후라이드치킨2: 0.9860 
-----------------------

🌟 훈련 완료! 총 소요 시간: 6분 2초
🏆 최적 검증 Macro F1: 0.9728
✅ 2차 Fine-tuning 완료


In [24]:
# 🧪 [2차 Fine-tuning 후 테스트] - 2차 학습 최적 모델 테스트

# 🚨🚨🚨 예측할 단일 이미지 파일의 로컬 경로를 여기에 입력하세요! 🚨🚨🚨
# 예시: test_image_path_2nd = "path/to/another/test/image.jpg"
test_image_path_2nd = "sample2.jpg"

print("\n" + "=" * 80)
print(f"🏆 2차 Fine-tuned 모델 최종 테스트: {os.path.basename(test_image_path_2nd)}")
print("=" * 80)

loaded_model_2nd = load_model_for_prediction(num_classes, model_path='efficientnet_fine_tuned_best_balanced.pth')

if loaded_model_2nd and os.path.exists(test_image_path_2nd):
    predicted_class_2nd, confidence_2nd = preprocess_and_predict(test_image_path_2nd, loaded_model_2nd,
                                                                 predict_transforms, device)
    if predicted_class_2nd != -1:
        pred_class_2nd = class_names[predicted_class_2nd]
        conf_percent_2nd = confidence_2nd * 100
        print(f"✅ 예측 클래스: {pred_class_2nd}")
        print(f"✅ 신뢰도: {conf_percent_2nd:.2f}%")
    else:
        print("⚠️ 예측 함수 실행 중 오류가 발생했습니다.")
elif not os.path.exists(test_image_path_2nd):
    print(f"⚠️ 테스트 이미지를 찾을 수 없습니다: {test_image_path_2nd}")
print("=" * 80)


🏆 2차 Fine-tuned 모델 최종 테스트: sample2.jpg
✅ 모델 로드 완료: efficientnet_fine_tuned_best_balanced.pth
✅ 예측 클래스: 후라이드치킨2
✅ 신뢰도: 51.06%
